In [1]:
print("dfff")

dfff


In [5]:
import json
import pandas as pd

# Reading the JSON files
with open('C:/Users/pasup/Desktop/etlproject/transaction_logs.json') as f:
    transactions = json.load(f)

with open('C:/Users/pasup/Desktop/etlproject/customers.json') as f:
    customers = json.load(f)

# Convert JSON data to DataFrame for easier processing
transactions_df = pd.DataFrame(transactions)
customers_df = pd.DataFrame(customers)



"""
print(transactions_df.head())
print(customers_df.head())

# EDA for Transactions
print("Transactions DataFrame Info:")
print(transactions_df.info())
print("Summary of Transactions DataFrame:")
print(transactions_df.describe())
print("Missing Values in Transactions DataFrame:")
print(transactions_df.isnull().sum())

# EDA for Customers
print("Customers DataFrame Info:")
print(customers_df.info())
print("Summary of Customers DataFrame:")
print(customers_df.describe())
print("Missing Values in Customers DataFrame:")
print(customers_df.isnull().sum()) """



# Convert datetime strings to datetime objects
transactions_df['datetime'] = pd.to_datetime(transactions_df['datetime'])

# Handle missing values
transactions_df.fillna({'failure_reason': 'No Failure'}, inplace=True)


import mysql.connector
from mysql.connector import Error

try:
    #Establish the connection to your existing MySQL database
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='root',
        database='revstore'  
    )

    if connection.is_connected():
        print("Connected to MySQL database")

        # Create a cursor object
        cursor = connection.cursor()

        # Insert data into customers table
        for i, row in customers_df.iterrows():
            cursor.execute('''
            INSERT INTO customers (customer_id, customer_name, country, city)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                customer_name=VALUES(customer_name),
                country=VALUES(country),
                city=VALUES(city);
            ''', tuple(row))

        # Insert data into transactions table
        for _, row in transactions_df.iterrows():
            cursor.execute('''
            INSERT INTO transactions (order_id, customer_id, product_id, product_name, product_category, 
                                      payment_type, qty, price, datetime, ecommerce_website_name, 
                                      payment_txn_id, payment_txn_success, failure_reason)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
                product_name=VALUES(product_name),
                product_category=VALUES(product_category),
                payment_type=VALUES(payment_type),
                qty=VALUES(qty),
                price=VALUES(price),
                datetime=VALUES(datetime),
                ecommerce_website_name=VALUES(ecommerce_website_name),
                payment_txn_id=VALUES(payment_txn_id),
                payment_txn_success=VALUES(payment_txn_success),
                failure_reason=VALUES(failure_reason);
            ''', tuple(row))

        # Commit the transaction
        connection.commit()

        print("Data inserted successfully into MySQL tables.") 

        # Fetch and display 5 records from the customers table
        cursor.execute("SELECT * FROM customers LIMIT 5;")
        customers = cursor.fetchall()
        print("\nCustomers Data:")
        for row in customers:
            print(row)

        # Fetch and display 5 records from the transactions table
        cursor.execute("SELECT * FROM transactions LIMIT 5;")
        transactions = cursor.fetchall()
        print("\nTransactions Data:")
        for row in transactions:
            print(row)

except Error as e:
    print(f"Error: {e}")




Connected to MySQL database
Data inserted successfully into MySQL tables.

Customers Data:
(102, 'Kenneth Singh', 'Sweden', 'Davidhaven')
(106, 'Mr. James Mendez', 'British Indian Ocean Territory (Chagos Archipelago)', 'Paulmouth')
(109, 'Joseph Martin', 'Somalia', 'Port Vickiemouth')
(135, 'Joann Romero', 'Timor-Leste', 'Port Raymondmouth')
(144, 'Robert Reeves', 'Wallis and Futuna', 'Lake Jason')

Transactions Data:
(1, 1731, 8609, 'Central home & kitche', 'Home & Kitchen', 'Wallet', 25, Decimal('64216.00'), datetime.datetime(2021, 7, 26, 20, 16, 14), 'www.bestbuy.com', 'a395e8c7', 'N', 'Card Expired')
(2, 5539, 2409, 'Upon sport', 'Sports', 'Card', 34, Decimal('69877.00'), datetime.datetime(2021, 8, 25, 3, 41, 32), 'www.flipkart.com', 'b40791f8', 'N', 'Invalid CVV')
(3, 3507, 1145, 'Across stationer', 'Stationery', 'Card', 28, Decimal('35491.00'), datetime.datetime(2024, 6, 21, 21, 39, 21), 'www.target.com', '7fb6cca9', 'N', 'Blocked Card')
(4, 4826, 8375, 'Start home & kitche', 'Ho